In [31]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/images'
print("Contents of data_dir:")
print(os.listdir(data_dir))
IMG_SIZE = (224, 224)  # Change from (300, 300) to (224, 224)  # efficient 300 else 224
BATCH_SIZE = 32
EPOCHS = 30

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contents of data_dir:
['NV', 'BCC', 'SCC', 'MEL']


In [32]:
import tensorflow as tf
from tensorflow import keras

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [33]:
# Print class indices
# print("Class indices:", train_generator.class_indices)
# model_saved = tf.keras.models.load_model('/content/drive/My Drive/models/skin_lesion_classifier_resnet50_final.keras')


model_saved = tf.keras.models.load_model('/content/drive/My Drive/models/skin_lesion_classifier_resnet50_final.keras',
                                         custom_objects={'F1Score': F1Score})
# Update class names to include SCC
class_names = ['MEL', 'NV', 'BCC', 'SCC']
for class_name in class_names:
    if not os.path.isdir(os.path.join(data_dir, class_name)):
        raise ValueError(f"Folder {class_name} not found in {data_dir}")

# Function to predict image
def predict_image(img_path, nv_threshold=0.7):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model_saved.predict(img_array)

    if np.argmax(prediction) == class_names.index('NV') and prediction[0][class_names.index('NV')] < nv_threshold:
        predicted_class = class_names[np.argsort(prediction[0])[-2]]
    else:
        predicted_class = class_names[np.argmax(prediction)]

    confidence = np.max(prediction)

    return predicted_class, confidence

finish_dir = '/content/drive/My Drive/finish'
# Interactive prediction loop


In [44]:
finish_dir = '/content/drive/My Drive/finish'
count = 0
j = 0

print("Contents of finish_dir:")
print(os.listdir(finish_dir))

for i in range(1, 54):
    image_found = False
    filename = f"{i}.jpg"
    if filename in os.listdir(finish_dir):
        img_path = os.path.join(finish_dir, filename)

        try:
            predicted_class, confidence = predict_image(img_path)
            if predicted_class == 'SCC':
                count += 1
            print(f"Image: {filename}")
            print(f"Predicted class: {predicted_class}")
            print(f"Confidence: {confidence:.2f}")
            print()
            j += 1
            image_found = True
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

    if not image_found:
        print(f"No image found with number {i}")
        break

if j > 0:
    print(f"accuracy ratio: {count/j:.2f}")
else:
    print("No images were successfully processed.")

print(f"Total corredct classifications: {count}")
print(f"Total images : {j}")
print("Thank you for using the classifier!")

Contents of finish_dir:
['BCC', 'MEL', 'SCC', 'NV', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Image: 1.jpg
Predicted class: MEL
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Image: 2.jpg
Predicted class: MEL
Confidence: 0.82

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Image: 3.jpg
Predicted class: MEL
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
Image: 4.jpg
Predicted class: SCC
Confidence: 0.80

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
Image: 5.jpg
Predicted class: MEL
Confidence: 0.99

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Image: 6.jpg
Predicted class: SCC
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
Image: 7.jpg
Predicted class: SCC
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Image: 8.jpg
Predicted class: SCC
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [35]:
print(count , j)

0 0


In [ ]:


##    OLD CODE

# # -*- coding: utf-8 -*-
# """check.ipynb

# Automatically generated by Colab.

# Original file is located at
#     https://colab.research.google.com/drive/1-oRzdlFRinkdaxDnwxg4gsSCNqAUw6-C
# """

# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.models import Model
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.metrics import Precision, Recall
# from tensorflow.keras import backend as K
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# data_dir = '/content/drive/My Drive/images'
# print("Contents of data_dir:")
# print(os.listdir(data_dir))
# IMG_SIZE = (300,300)  # efficient 300 else 224
# BATCH_SIZE = 32
# EPOCHS = 30

# import tensorflow as tf
# from tensorflow import keras

# class F1Score(tf.keras.metrics.Metric):
#     def __init__(self, name='f1_score', **kwargs):
#         super().__init__(name=name, **kwargs)
#         self.precision = tf.keras.metrics.Precision()
#         self.recall = tf.keras.metrics.Recall()

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         self.precision.update_state(y_true, y_pred, sample_weight)
#         self.recall.update_state(y_true, y_pred, sample_weight)

#     def result(self):
#         p = self.precision.result()
#         r = self.recall.result()
#         return 2 * ((p * r) / (p + r + keras.backend.epsilon()))

#     def reset_state(self):
#         self.precision.reset_state()
#         self.recall.reset_state()

#     def get_config(self):
#         base_config = super().get_config()
#         return base_config

# # Print class indices
# # print("Class indices:", train_generator.class_indices)
# model_saved = tf.keras.models.load_model('/content/drive/My Drive/models/skin_lesion_classifier_resnet50_final.keras')
# # Update class names to include SCC
# class_names = ['MEL', 'NV', 'BCC', 'SCC']
# for class_name in class_names:
#     if not os.path.isdir(os.path.join(data_dir, class_name)):
#         raise ValueError(f"Folder {class_name} not found in {data_dir}")

# # Function to predict image
# def predict_image(img_path, nv_threshold=0.7):
#     img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
#     img_array = tf.keras.preprocessing.image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     img_array /= 255.0

#     prediction = model_saved.predict(img_array)

#     if np.argmax(prediction) == class_names.index('NV') and prediction[0][class_names.index('NV')] < nv_threshold:
#         predicted_class = class_names[np.argsort(prediction[0])[-2]]
#     else:
#         predicted_class = class_names[np.argmax(prediction)]

#     confidence = np.max(prediction)

#     return predicted_class, confidence

# finish_dir = '/content/drive/My Drive/finish'
# # Interactive prediction loop

# count=0
# j=0
# for i in range(1,54):
#     user_input=i
#     try:
#         image_number = int(user_input)

#         for filename in os.listdir(finish_dir):
#             if filename.startswith(f"{image_number}.") and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#                 img_path = os.path.join(finish_dir, filename)

#                 predicted_class, confidence = predict_image(img_path)
#                 if(predicted_class=='MEL'):
#                     count=count+1
#                 print(f"Image: {filename}")
#                 print(f"Predicted class: {predicted_class}")
#                 print(f"Confidence: {confidence:.2f}")
#                 print()
#                 j+=1
#                 break
#         else:
#             print(f"No image found with number {image_number}")
#             break

#     except ValueError:
#         print("Invalid input")

# print(count/j)
# print("Thank you for using the classifier!")

# print(count , j)